In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('preprocessed_train.csv')
df_test = pd.read_csv('preprocessed_test.csv')

In [3]:
df.head()

,Month,Age,Profession,Income_Annual,Base_Salary_PerMonth,Total_Bank_Accounts,Total_Credit_Cards,Rate_Of_Interest,Total_Current_Loans,Delay_from_due_date,...,Current_Debt_Outstanding,Ratio_Credit_Utilization,Credit_History_Age,Payment_of_Min_Amount,Per_Month_EMI,Monthly_Investment,Payment_Behaviour,Monthly_Balance,Credit_Score,Loan_Count
0,7,51,11,101583.48,8648.290000,5,7,10,4,8,...,50.93,34.462154,289,0,190.811017,630.015789,3,314.002193,1,4
1,1,23,14,101926.95,8635.912500,4,4,9,1,13,...,1058.00,39.693812,245,0,70.587681,662.803927,4,410.199642,1,1
2,2,49,14,158871.12,12962.260000,0,4,8,1,8,...,576.48,39.367225,228,0,86.905860,746.805985,4,742.514154,1,1
3,6,40,3,60379.28,4804.606667,5,6,18,3,15,...,725.39,29.061701,205,0,90.906385,166.418658,1,473.135623,1,3
4,5,17,0,50050.83,4085.902500,9,10,20,5,28,...,3419.10,30.386321,54,1,190.445060,56.789441,0,401.355749,0,5


In [4]:
X = df.drop('Credit_Score', axis=1)
y = df['Credit_Score']
X_test = df_test.drop('ID', axis=1)

In [5]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Define the best parameters from Optuna
best_params = {
    'n_estimators': 299,
    'max_depth': 14,
    'learning_rate': 0.09029244155247403,
    'subsample': 0.8621963214294011,
    'colsample_bytree': 0.5094283190411915,
    'gamma': 0.06544346239673754,
    'min_child_weight': 1,
    'reg_alpha': 1.589281578937299e-07,
    'reg_lambda': 1.7694621194522904e-05,
    'max_delta_step': 1,
    'objective': 'multi:softmax',
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'random_state': 42
}

# Train the final model with the best parameters
final_model = XGBClassifier(**best_params)
final_model.fit(X, y)

# Make predictions on the training data
y_pred = final_model.predict(X)

# Calculate and print the accuracy on the training data
accuracy_test = accuracy_score(y, y_pred)
print("Training accuracy:", accuracy_test)

Training accuracy: 1.0


In [6]:
y_test = final_model.predict(X_test)

In [7]:
y_pred_test = pd.Series(y_test).map({0: 'Poor', 1: 'Standard', 2: 'Good'})
final = pd.DataFrame({'ID': df_test['ID'], 'Credit_Score': y_pred_test})
final.head()

,ID,Credit_Score
0,0x2145,Standard
1,0x7d59,Poor
2,0xe753,Poor
3,0x19813,Standard
4,0x1dc5e,Standard


In [8]:
final.to_csv('final_submission.csv', index=False)